# Setup

In [101]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from generative_social_choice.utils.helper_functions import get_base_dir_path

In [103]:
from generative_social_choice.slates.voting_algorithms import (
    SequentialPhragmenMinimax,
    GreedyTotalUtilityMaximization,
    ExactTotalUtilityMaximization,
    LPTotalUtilityMaximization,
    VotingAlgorithm,
    GeometricTransformation,
)
from generative_social_choice.slates.voting_algorithm_axioms import (
    IndividualParetoAxiom,
    HappiestParetoAxiom,
    CoverageAxiom,
    MinimumAndTotalUtilityParetoAxiom,
    VotingAlgorithmAxiom,
    NonRadicalMinUtilityAxiom,
    NonRadicalTotalUtilityAxiom,
)
from generative_social_choice.test.utilities_for_testing import rated_vote_cases

# Load Data

In [104]:
voting_algorithm_evals_dir = get_base_dir_path() / "data" / "voting_algorithm_evals"
latest = True
if latest:
    file = sorted(voting_algorithm_evals_dir.glob("*.csv"))[-1]
else:
    file = voting_algorithm_evals_dir / "2025-01-20-180945.csv"
file

df = pd.read_csv(file, index_col=0, header=[0, 1])

In [105]:
# Select subset of rows
# df = df.iloc[-8:]
df

vote                                                       Simple 1  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=None)              1.0   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=Ge...              1.0   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=Ge...              1.0   
Phragmen(marginal_slate, clear=True, redist=True)               1.0   
Phragmen(marginal_slate, clear=True, redist=False)              1.0   
Phragmen(marginal_slate, clear=False, redist=True)              1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...              1.0   
Phragmen(marginal_previous, clear=True, redist=...              1.0   
Phragmen(marginal_previous, clear=True, redist=...              1.0   
Phragmen(marginal_previous, clear=False, redist...              1.0   
Phragmen(marginal_previous, clear=False, redist...              1.0   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=None)                                    1.0   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                                    1.0   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                                    1.0   
Phragmen(marginal_slate, clear=True, redist=True)                                     1.0   
Phragmen(marginal_slate, clear=True, redist=False)                                    1.0   
Phragmen(marginal_slate, clear=False, redist=True)                                    1.0   
Phragmen(marginal_slate, clear=False, redist=Fa...                                    1.0   
Phragmen(marginal_previous, clear=True, redist=...                                    1.0   
Phragmen(marginal_previous, clear=True, redist=...                                    1.0   
Phragmen(marginal_previous, clear=False, redist...                                    1.0   
Phragmen(marginal_previous, clear=False, redist...                                    1.0   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=None)                          1.0   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=Ge...                          1.0   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=Ge...  

## Overall Performance

In [106]:
df.sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                               144.333333
ExactTotalUtilityMaximization(utility_transform=None)                                157.750000
LPTotalUtilityMaximization(utility_transform=None)                                   152.791667
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))     146.666667
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))      161.375000
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))         153.791667
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))    142.791667
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))     155.083333
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))        154.208333
Phragmen(marginal_slate, clear=True, redist=True)                                    141.375000
Phragmen(marginal_slate, clear=True, red

In [107]:
pd.DataFrame(np.trunc(df.values), index=df.index, columns=df.columns).sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                               128.0
ExactTotalUtilityMaximization(utility_transform=None)                                154.0
LPTotalUtilityMaximization(utility_transform=None)                                   149.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))     133.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))      152.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))         152.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))    125.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))     145.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0))        152.0
Phragmen(marginal_slate, clear=True, redist=True)                                    120.0
Phragmen(marginal_slate, clear=True, redist=False)                                   119.0

## Differential Test Performance

In [108]:
non_uniform_columns = df.loc[:, df.nunique() > 1]
non_uniform_columns


vote                                                       Simple 3  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=None)         1.000000   
GreedyTotalUtilityMaximization(utility_transfor...         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...         1.000000   
GreedyTotalUtilityMaximization(utility_transfor...         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...         1.000000   
Phragmen(marginal_slate, clear=True, redist=True)          0.541667   
Phragmen(marginal_slate, clear=True, redist=False)         0.291667   
Phragmen(marginal_slate, clear=False, redist=True)         0.375000   
Phragmen(marginal_slate, clear=False, redist=Fa...         0.250000   
Phragmen(marginal_previous, clear=True, redist=...         0.416667   
Phragmen(marginal_previous, clear=True, redist=...         0.458333   
Phragmen(marginal_previous, clear=False, redist...         0.500000   
Phragmen(marginal_previous, clear=False, redist...         0.500000   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                               1.000000   
ExactTotalUtilityMaximization(utility_transform...                               1.000000   
LPTotalUtilityMaximization(utility_transform=None)                               1.000000   
GreedyTotalUtilityMaximization(utility_transfor...                               1.000000   
ExactTotalUtilityMaximization(utility_transform...                               1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...                               1.000000   
GreedyTotalUtilityMaximization(utility_transfor...                               1.000000   
ExactTotalUtilityMaximization(utility_transform...                               1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...                               1.000000   
Phragmen(marginal_slate, clear=True, redist=True)                                0.416667   
Phragmen(marginal_slate, clear=True, redist=False)                               0.250000   
Phragmen(marginal_slate, clear=False, redist=True)                               0.291667   
Phragmen(marginal_slate, clear=False, redist=Fa...                               0.333333   
Phragmen(marginal_previous, clear=True, redist=...                               0.500000   
Phragmen(marginal_previous, clear=True, redist=...                               0.416667   
Phragmen(marginal_previous, clear=False, redist...                               0.583333   
Phragmen(marginal_previous, clear=False, redist...                               0.458333   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                     1.000000   
ExactTotalUtilityMaximization(utility_transform...                     1.000000   
LPTotalUtilityMaximization(utility_transform=None)                     1.000000   
GreedyTotalUtilityMaximization(utility_transfor...                     1.000000   
ExactTotalUtilityMaximization(utility_transform...                     1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...                     1.000000   
GreedyTotalUtilityMaximization(utility_transfor...                     1.000000   
ExactTotalUtilityMaximization(utility_transform...                     1.000000   
LPTotalUtilityMaximization(utility_transform=Ge...  

These counts show which test cases and which axioms most frequently show unique behavior across the `VotingAlgorithm`s

In [109]:
# Count the number of columns with each name in both levels of the MultiIndex
level_0_counts = non_uniform_columns.columns.get_level_values(0).value_counts()
level_1_counts = non_uniform_columns.columns.get_level_values(1).value_counts()

print("Counts for level 0:")
print(level_0_counts)



Counts for level 0:
vote
Simple 3           6
Ex A.1             6
Ex Alg1.5          6
Ex 1.3             6
Ex 3.1             6
Ex 4.2             6
Ex Alg1.3          6
Ex 4.3             5
Ex D.1             5
Ex 1.1 modified    4
Ex Alg A.2         4
Ex Alg A.1         4
Ex C.2             4
Ex B.3             3
Ex 4.1             1
Ex 2.1             1
Ex Alg1.4          1
Ex Alg2.1          1
Name: count, dtype: int64


In [110]:
print("\nCounts for level 1:")
print(level_1_counts)


Counts for level 1:
subtest
Non-radical Total Utility Pareto Efficiency            16
Non-radical Minimum Utility Pareto Efficiency          15
Maximum Coverage                                       14
Minimum Utility and Total Utility Pareto Efficiency    13
m-th Happiest Person Pareto Efficiency                 10
Individual Pareto Efficiency                            7
Name: count, dtype: int64


## Axiom Pass Rate

In [111]:
df.groupby(df.columns.get_level_values(1), axis=1).mean()

C:\Users\aaron\AppData\Local\Temp\ipykernel_20540\1810388850.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df.groupby(df.columns.get_level_values(1), axis=1).mean()


subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Minimum Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),1.000000,0.865741,0.876543,0.876543,0.726852,1.000000
ExactTotalUtilityMaximization(utility_transform=None),0.998457,0.993827,0.998457,1.000000,0.851852,1.000000
LPTotalUtilityMaximization(utility_transform=None),1.000000,0.962963,0.962963,0.961420,0.810185,0.961420
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,0.868827,0.865741,0.861111,0.847222,0.989198
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),0.998457,0.998457,0.992284,0.996914,0.995370,0.995370
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.000000,0.933642,0.945988,0.935185,0.939815,0.941358
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),0.969136,0.867284,0.856481,0.830247,0.825617,0.939815
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),0.975309,0.961420,0.947531,0.939815,0.959877,0.959877
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),1.000000,0.938272,0.941358,0.939815,0.941358,0.950617
"Phragmen(marginal_slate, clear=True, redist=True)",0.938272,0.881173,0.887346,0.763889,0.850309,0.915123


# Debuggign

In [193]:
# alg = GreedyTotalUtilityMaximization()
alg = ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))

In [194]:
df1 = df.loc[str(alg),:].unstack()
df1

subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Minimum Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
vote,,,,,,
Ex 1.1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Ex 1.1 modified,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Ex 1.2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Ex 1.3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Ex 2.1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Ex 2.2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Ex 3.1,1.000000,0.958333,0.875000,0.916667,0.916667,0.958333
Ex 4.1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Ex 4.2,0.958333,1.000000,0.958333,1.000000,1.000000,1.000000


In [207]:
# case = rated_vote_cases["Ex 1.1 modified"]
# case = rated_vote_cases["Ex 3.1"]
case = rated_vote_cases["Ex 4.3"]
axiom = HappiestParetoAxiom()
# axiom = MinimumAndTotalUtilityParetoAxiom()
# axiom = NonRadicalTotalUtilityAxiom()

case.rated_votes

,s1,s2,s3,s4,s5
0,5,0,0,0,1
1,5,5,0,0,1
2,0,5,0,0,1
3,0,0,5,0,1
4,0,0,5,5,1
5,0,0,0,5,1


In [208]:
df.loc[:,df.columns.get_level_values(1) == axiom.name]
# df.columns.get_level_values(1) == type(axiom).__name__
# df.columns.get_level_values(1), axiom.name


vote,Simple 1,Simple 2,Simple 3,Ex 1.1,Ex 1.1 modified,Ex 1.2,Ex A.1,Ex 1.3,Ex 2.1,Ex 2.2,...,Ex B.3,Ex C.1,Ex C.2,Ex D.1,Ex Alg1.3,Ex Alg1.4,Ex Alg1.5,Ex Alg2.1,Ex Alg A.1,Ex Alg A.2
subtest,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,...,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency,m-th Happiest Person Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),1.0,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0
ExactTotalUtilityMaximization(utility_transform=None),1.0,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0
LPTotalUtilityMaximization(utility_transform=None),1.0,1.0,1.000000,1.0,0.958333,1.0,1.000000,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.0,1.0,1.000000,1.0,1.000000,1.0,0.791667,0.916667,1.0,1.0,...,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.0,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.0,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0
GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),1.0,1.0,1.000000,1.0,1.000000,1.0,0.291667,0.416667,1.0,1.0,...,1.0,1.0,1.0,0.791667,1.000000,1.0,1.000000,1.0,1.0,1.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),1.0,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0
LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=10.0)),1.0,1.0,1.000000,1.0,1.000000,1.0,1.000000,1.000000,1.0,1.0,...,1.0,1.0,1.0,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0


In [209]:
alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)[0]

['s2', 's3', 's4']

In [210]:
axiom.satisfactory_slates(rated_votes=case.rated_votes, slate_size=case.slate_size)

[('s1', 's2', 's3'),
 ('s1', 's2', 's4'),
 ('s1', 's2', 's5'),
 ('s1', 's3', 's4'),
 ('s1', 's3', 's5'),
 ('s1', 's4', 's5'),
 ('s2', 's3', 's4'),
 ('s2', 's3', 's5'),
 ('s2', 's4', 's5'),
 ('s3', 's4', 's5')]

In [211]:
num_aug_cases = 24
aug_case_votes = pd.DataFrame(index = range(num_aug_cases), columns=["rated_votes", "assignments", "axiom_slates", "alg_slate", "satisfied"])
for i, aug_case in enumerate(case.augmented_cases[:num_aug_cases]):
    # print(aug_case)
    aug_case_votes.at[i, "rated_votes"] = aug_case
    aug_case_votes.at[i, "axiom_slates"] = axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "alg_slate"], aug_case_votes.at[i, "assignments"] = alg.vote(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "satisfied"] = axiom.evaluate_assignment(
        rated_votes=aug_case,
        assignments=aug_case_votes.at[i, "assignments"],
        slate_size=case.slate_size,
    )
    # print(axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size))
# aug_case_votes.iloc[:,2:]

In [212]:
failures = aug_case_votes[aug_case_votes["satisfied"] == False]
failures.iloc[:,2:]


,axiom_slates,alg_slate,satisfied
7,"[(s1, s2, s3), (s1, s2, s4), (s1, s2, s5), (s1...","[s1, s2, s4]",False


In [213]:
f0 = failures.iloc[0]
f0.rated_votes
# (f0.rated_votes - case.rated_votes)*1e6


,s1,s2,s3,s4,s5
0,5.00011,0.000321,0.000136,0.0002,1.000379
1,5.00011,5.000321,0.000136,0.0002,1.000379
2,0.00011,5.000321,0.000136,0.0002,1.000379
3,0.00011,0.000321,5.000136,0.0002,1.000379
4,0.00011,0.000321,5.000136,5.0002,1.000379
5,0.00011,0.000321,0.000136,5.0002,1.000379


In [214]:
((f0.rated_votes - case.rated_votes)*1e6)

,s1,s2,s3,s4,s5
0,109.525354,320.571887,135.707746,200.302462,379.156609
1,109.525354,320.571887,135.707746,200.302462,379.156609
2,109.525354,320.571887,135.707746,200.302462,379.156609
3,109.525354,320.571887,135.707746,200.302462,379.156609
4,109.525354,320.571887,135.707746,200.302462,379.156609
5,109.525354,320.571887,135.707746,200.302462,379.156609


In [215]:
f0.assignments

,candidate_id
0,s1
1,s2
2,s2
3,s2
4,s4
5,s4


In [216]:
alg.vote(rated_votes=f0.rated_votes, slate_size=case.slate_size)

(['s1', 's2', 's4'],
   candidate_id
 0           s1
 1           s2
 2           s2
 3           s2
 4           s4
 5           s4)

In [219]:
axiom.evaluate_assignment(
    rated_votes=f0.rated_votes,
    assignments=f0.assignments,
    slate_size=case.slate_size,
)

False

In [218]:
from generative_social_choice.slates.voting_algorithm_axioms import VotingAlgorithmAxiom
from kiwiutils.kiwilib import leafClasses
[cls.__name__ for cls in leafClasses(VotingAlgorithmAxiom)]

['MinimumAndTotalUtilityParetoAxiom',
 'HappiestParetoAxiom',
 'NonRadicalMinUtilityAxiom',
 'IndividualParetoAxiom',
 'NonRadicalTotalUtilityAxiom',
 'CoverageAxiom']